In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os


/Users/wenxixia/opt/anaconda3/lib/python3.8/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
start_date ='2021-11-05'
end_date ='2021-11-11'
tokens = ['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV',
       'AAVE', 'COMP']
connection_string = "sqlite:////Users/wenxixia/git-intro/UCB-MFE-python-preprogram/Lectures/Lecture 4/data/data.db"

In [3]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [4]:
for token in tokens:
    print(f'running for {token} for date {start_date} to {end_date}', end='\r')
   
    res = pm.execute_notebook(
        'market_analysis.ipynb',
        f'{base_output_folder}/market_analysis_{token}.ipynb',
        parameters = {
            'start_date': start_date,
            'end_date': end_date,
            'token': token,
            'connection_string': connection_string,
        }
    )
    

In [5]:
outputs = sb.read_notebooks('outputs')
summary = pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'volatility_AVG': scrap['prices'].data.volatility.mean(),
        'return_AVG': scrap['prices'].data['return'].mean(),
        'max_dropdown': scrap['prices'].data['maxdropdown'].max(),
        'range_last_close_minus_first_close':abs(float(scrap['prices'].data['close'].head(1))-\
                                            float(scrap['prices'].data['close'].tail(1))),
        }) for book, scrap in outputs.notebook_scraps.items()])
print(summary)

   token  volatility_AVG  return_AVG  max_dropdown  \
0   AAVE        0.138614   -0.000385      0.113934   
1    ADA        0.141206    0.000504      0.122552   
2   ATOM        0.155429   -0.000968      0.199680   
3   AVAX        0.253301    0.000631      0.123461   
4    BTC        0.086268    0.000430      0.063230   
5   COMP        0.135639   -0.000682      0.138580   
6    CRV        0.218442   -0.000039      0.206905   
7    DOT        0.141692   -0.000836      0.156088   
8    ETH        0.091043    0.000180      0.060839   
9    SOL        0.161119   -0.000347      0.117807   
10  USDT        0.003029   -0.000001      0.001398   

    range_last_close_minus_first_close  
0                              19.5960  
1                               0.1343  
2                               5.1600  
3                               6.1900  
4                            3715.3500  
5                              36.1700  
6                               0.0914  
7                      